# TF mnist

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define different `commandComponent` using dsl.command_component decorator
- Create `pipeline` using components defined by dsl.command_component

**Motivations** - This notebook explains how to run a pipeline with distributed training component.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient, dsl

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 2. Define command component

We defined sample component using dsl.command_component in [dsl_component.py](dsl_component.py).

In [ ]:
with open("dsl_component.py") as fin:
    print(fin.read())

In [ ]:
%load_ext autoreload
%autoreload 2

from dsl_component import train_tf
help(train_tf)

# 3. Pipeline job
## 3.1 Build pipeline

In [ ]:
@dsl.pipeline(
    description="Train using TF component",
)
def sample_pipeline():
    tf_job = train_tf(epochs=1)
    tf_job.compute = "cpu-cluster"
    tf_job.resources.instance_count=2
    tf_job.outputs.trained_model_output.mode = "upload"

# create pipeline instance
pipeline = sample_pipeline()


## 3.2 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="tf_mnist")
print(f'Job link: {pipeline_job.services["Studio"].endpoint}')

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)